In [1]:
from datasets import load_dataset
from collections import defaultdict
ds = load_dataset('hotpotqa/hotpot_qa', 'fullwiki', trust_remote_code=True)

/home/junseokkim00/anaconda3/envs/rp2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## generate w/o_distractor data

In [2]:
def remove_distractor(example):
    context = defaultdict(list)
    context['title']=[]
    context['sentences']=[]
    for title in example['supporting_facts']['title']:
        idx = example['context']['title'].index(title)
        if title == example['context']['title'][idx]:
            context['title'].append(title)
            context['sentences'].append(example['context']['sentences'][idx])
        else:
            print(f"{title} is not inside the context!")
    example['context'] = context
    return example
ds_train_rm_distract = ds['train'].map(remove_distractor)

In [3]:
# ds_train_rm_distract
import random

k=300
indices = random.sample(range(len(ds_train_rm_distract)), k)
test_dataset = ds_train_rm_distract.select(indices)


In [7]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from dotenv import find_dotenv, load_dotenv
# from LLMs import set_model
import os

load_dotenv(find_dotenv())
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model='llama3-8b-8192')
# llm = set_model('gpt-4o-mini')

SyntaxError: invalid syntax (3899325849.py, line 4)

In [5]:
human_msg="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Keep the answer concise and output in a short answer form.

Question: {question} 

Context: {context} 

Answer:
"""
total_cnt=0
cnt=0
pred=[]
labels=[]
for data in test_dataset:
    question, answer, sentences = data['question'], data['answer'], data['context']['sentences']
    print(f"{total_cnt+1}/{len(test_dataset)} Question: {question}")
    context=""
    for sent_ls in sentences:
        context+=" ".join(sent_ls)
        context+='\n\n'
    chat_prompt = ChatPromptTemplate.from_messages([
        ("user", human_msg)
    ])
    chain = chat_prompt | llm
    output = chain.invoke({
        'question': question,
        'context': context
    })
    print(output.content)
    msg = human_msg + output.content + 'Now, please provide only the answer without any explanation or preambles. '
    answer_extraction = ChatPromptTemplate.from_messages([
        ("user", msg)
    ])
    chain_ans = answer_extraction | llm
    final_output = chain_ans.invoke({
        'question': question,
        'context': context
    })
    print(final_output.content, answer)
    pred.append(final_output.content)
    labels.append(answer)
    if final_output.content == answer:
        cnt+=1
    total_cnt+=1

print(f"exact_match: {cnt} / {total_cnt}")
    
    


1/300 Question: Yang Seung-Pil, is a South Korean actor, he has two supporting roles in the historical television series "Maids" (2015), in English language literature sageuk usually refers to historical films and television series, of which location?
South Korea.
South Korea South Korea
2/300 Question: Which British national museum holds Over The Top and banches at five locations in England?
The Imperial War Museum, London.
Imperial War Museum, London. Imperial War Museum
3/300 Question: The Double EP: A Sea of Split Peas is an EP by which Australian singer, songwriter, and musician?
Courtney Barnett.
Courtney Barnett Courtney Melba Barnett
4/300 Question: What media organization which posts talks is Jesse Dylan a member of?
Jesse Dylan is a member of TED, a media organization that posts talks online for free distribution.
TED TED
5/300 Question: Oh Yeon-seo is one of the former members of the group LUV, which stands for Love yoUR Voice, is what kind of group?
K-pop girls group.
K-pop

In [5]:
cnt=0
total_cnt=0
for p, l in zip(pred, labels):
    if p.lower() == l.lower():
        cnt+=1
    total_cnt+=1
cnt, total_cnt

NameError: name 'pred' is not defined

## check data with distractors

In [7]:
test_ds = ds['train'].select(indices)

In [8]:
human_msg="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Keep the answer concise and output in a short answer form.

Question: {question} 

Context: {context} 

Answer:
"""
total_cnt=0
cnt=0
pred_og=[]
labels_og=[]
for data in test_ds:
    question, answer, sentences = data['question'], data['answer'], data['context']['sentences']
    print(f"{total_cnt+1}/{len(test_ds)} Question: {question}")
    context=""
    for sent_ls in sentences:
        context+=" ".join(sent_ls)
        context+='\n\n'
    chat_prompt = ChatPromptTemplate.from_messages([
        ("user", human_msg)
    ])
    chain = chat_prompt | llm
    output = chain.invoke({
        'question': question,
        'context': context
    })
    print(output.content)
    msg = human_msg + output.content + 'Now, please provide only the answer without any explanation or preambles. '
    answer_extraction = ChatPromptTemplate.from_messages([
        ("user", msg)
    ])
    chain_ans = answer_extraction | llm
    final_output = chain_ans.invoke({
        'question': question,
        'context': context
    })
    print(final_output.content, answer)
    pred_og.append(final_output.content)
    labels_og.append(answer)
    if final_output.content == answer:
        cnt+=1
    total_cnt+=1

print(f"exact_match: {cnt} / {total_cnt}")
    
    


1/300 Question: Yang Seung-Pil, is a South Korean actor, he has two supporting roles in the historical television series "Maids" (2015), in English language literature sageuk usually refers to historical films and television series, of which location?
The location of a sageuk (historical drama) film or television series, as referred to in English language literature, is typically South Korea.
South Korea. South Korea
2/300 Question: Which British national museum holds Over The Top and banches at five locations in England?
The British national museum that holds "Over The Top" and has branches at five locations in England is the Imperial War Museum.
Imperial War Museum. Imperial War Museum
3/300 Question: The Double EP: A Sea of Split Peas is an EP by which Australian singer, songwriter, and musician?
The Double EP: A Sea of Split Peas is an EP by Australian singer, songwriter, and musician Courtney Barnett.
Courtney Barnett Courtney Melba Barnett
4/300 Question: What media organization 

In [9]:
cnt=0
total_cnt=0
for p, l in zip(pred_og, labels_og):
    if p.lower() == l.lower():
        cnt+=1
    total_cnt+=1
cnt, total_cnt

(99, 300)

## Only supporting facts

In [10]:
human_msg="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Keep the answer concise and output in a short answer form.

Question: {question} 

Context: {context} 

Answer:
"""
total_cnt=0
cnt=0
pred_sf=[]
labels_sf=[]
for data in test_dataset:
    question, answer, sentences = data['question'], data['answer'], data['context']['sentences']
    title, sent_id = data['supporting_facts']['title'], data['supporting_facts']['sent_id']
    print(f"{total_cnt+1}/{len(test_dataset)} Question: {question}")
    context=""
    for sent_ls, id in zip(sentences, sent_id):
        context+=sent_ls[id]
        context+='\n\n'
    chat_prompt = ChatPromptTemplate.from_messages([
        ("user", human_msg)
    ])
    chain = chat_prompt | llm
    output = chain.invoke({
        'question': question,
        'context': context
    })
    print(output.content)
    msg = human_msg + output.content + 'Now, please provide only the answer without any explanation or preambles. '
    answer_extraction = ChatPromptTemplate.from_messages([
        ("user", msg)
    ])
    chain_ans = answer_extraction | llm
    final_output = chain_ans.invoke({
        'question': question,
        'context': context
    })
    print(final_output.content, answer)
    pred_sf.append(final_output.content)
    labels_sf.append(answer)
    if final_output.content == answer:
        cnt+=1
    total_cnt+=1

print(f"exact_match: {cnt} / {total_cnt}")
    
    


1/300 Question: Yang Seung-Pil, is a South Korean actor, he has two supporting roles in the historical television series "Maids" (2015), in English language literature sageuk usually refers to historical films and television series, of which location?
Sageuk refers to historical films and television series of South Korea.
South Korea. South Korea
2/300 Question: Which British national museum holds Over The Top and banches at five locations in England?
Imperial War Museums (IWM)
Imperial War Museums (IWM) Imperial War Museum
3/300 Question: The Double EP: A Sea of Split Peas is an EP by which Australian singer, songwriter, and musician?
Courtney Barnett.
Courtney Barnett Courtney Melba Barnett
4/300 Question: What media organization which posts talks is Jesse Dylan a member of?
Jesse Dylan is a member of TED, a media organization that posts talks online for free distribution.
Jesse Dylan is a member of TED. TED
5/300 Question: Oh Yeon-seo is one of the former members of the group LUV, w

The Imperial Palace Hotel and Casino, now known as The Linq, had 2,640 rooms.
2,640 2,640-room
13/300 Question: Which actress starred in "Perfect High" and currents stars in the television series "Famous in Love"?
Bella Thorne.
Bella Thorne. Bella Thorne
14/300 Question: Which German tennis player holding 13 Masters Series titles and an Olympic gold medal attended the 1997 Wimbledon Championships in London? 
Boris Becker.
Boris Becker Boris Becker
15/300 Question: Are South of the Border and The Elephant in the Living Room from the same country?
Yes, both South of the Border and The Elephant in the Living Room are American documentary films.
Yes yes
16/300 Question: Erik Kramer was a starter for the Detroit Lions after injuries sustained by the quarterback who played college ball for what team?
Rodney Peete played college ball for the USC Trojans.
USC Trojans USC Trojans
17/300 Question: Which singer is younger? Marty Raybon or Martin Fry?
Martin Fry (born 1958) is older than Marty Ray

In [11]:
cnt=0
total_cnt=0
for p, l in zip(pred_sf, labels_sf):
    if p.lower() == l.lower():
        cnt+=1
    total_cnt+=1
cnt, total_cnt

(142, 300)